# **SETUP**

First we need to install duckdb, all other libraries (e.g., sqlite, urllib,zipfile,pandas,...) used in our tutorial are already installed in the google cloud.


In [ ]:
!pip install duckdb --pre

Now we download the Voter dataset and decompress it.

In [ ]:
import urllib.request
import zipfile
import os

print("Downloading Dataset")
urllib.request.urlretrieve("https://zenodo.org/record/4106028/files/NC%20Voters.zip?download=1", "ncvoter.tsv.zip")

print("Decompressing files")
with zipfile.ZipFile("ncvoter.tsv.zip","r") as zip_ref:
	zip_ref.extractall("./")

print("Finished.")


# **Loading The Data**

We will first load the data in Pandas and measure how long it takes.

In [ ]:
import time
import pandas

start = time.time()

ncvoters_pandas = pandas.read_csv('ncvoter_sample.tsv', sep='\t', low_memory=False)
precinctvotes_pandas = pandas.read_csv('precinct_votes.tsv', sep='\t', low_memory=False)
end = time.time()
print("Loading data in Pandas : " +str(end - start))

We do the same for SQLite

In [ ]:
import sqlite3
import os
import csv

CREATE_NCVOTERS_SQL = '''
    CREATE TABLE IF NOT EXISTS ncvoters(county_id STRING, county_desc STRING, voter_reg_num STRING,status_cd STRING, voter_status_desc STRING, reason_cd STRING, voter_status_reason_desc STRING, absent_ind STRING, name_prefx_cd STRING,last_name STRING, first_name STRING, midl_name STRING, name_sufx_cd STRING, full_name_rep STRING,full_name_mail STRING, house_num STRING, half_code STRING, street_dir STRING, street_name STRING, street_type_cd STRING, street_sufx_cd STRING, unit_designator STRING, unit_num STRING, res_city_desc STRING,state_cd STRING, zip_code STRING, res_street_address STRING, res_city_state_zip STRING, mail_addr1 STRING, mail_addr2 STRING, mail_addr3 STRING, mail_addr4 STRING, mail_city STRING, mail_state STRING, mail_zipcode STRING, mail_city_state_zip STRING, area_cd STRING, phone_num STRING, full_phone_number STRING, drivers_lic STRING, race_code STRING, race_desc STRING, ethnic_code STRING, ethnic_desc STRING, party_cd STRING, party_desc STRING, sex_code STRING, sex STRING, birth_age INTEGER, birth_place STRING, registr_dt STRING, precinct_abbrv STRING, precinct_desc STRING,municipality_abbrv STRING, municipality_desc STRING, ward_abbrv STRING, ward_desc STRING, cong_dist_abbrv STRING, cong_dist_desc STRING, super_court_abbrv STRING, super_court_desc STRING, judic_dist_abbrv STRING, judic_dist_desc STRING, nc_senate_abbrv STRING, nc_senate_desc STRING, nc_house_abbrv STRING, nc_house_desc STRING,county_commiss_abbrv STRING, county_commiss_desc STRING, township_abbrv STRING, township_desc STRING,school_dist_abbrv STRING, school_dist_desc STRING, fire_dist_abbrv STRING, fire_dist_desc STRING, water_dist_abbrv STRING, water_dist_desc STRING, sewer_dist_abbrv STRING, sewer_dist_desc STRING, sanit_dist_abbrv STRING, sanit_dist_desc STRING, rescue_dist_abbrv STRING, rescue_dist_desc STRING, munic_dist_abbrv STRING, munic_dist_desc STRING, dist_1_abbrv STRING, dist_1_desc STRING, dist_2_abbrv STRING, dist_2_desc STRING, confidential_ind STRING, age STRING, ncid STRING, vtd_abbrv STRING, vtd_desc STRING);
'''

CREATE_PRECINCTVOTES_SQL = '''
CREATE TABLE IF NOT EXISTS precinct_votes(county STRING, precinct STRING, total_votes INT, romney_percentage DOUBLE);
'''

start = time.time()

if os.path.isfile("voters_sqlite.db"): #Checks if sqlite database already exits
		os.system("rm voters_sqlite.db") #If Exists deletes Database
db_sqlite = sqlite3.connect('voters_sqlite.db') #Connects to a SQLite database instance called voters_sqlite.db
sqlite_cursor = db_sqlite.cursor() # Creates a Cursor, that allows us to issue Create Table Stataments
sqlite_cursor.execute(CREATE_NCVOTERS_SQL)
sqlite_cursor.execute(CREATE_PRECINCTVOTES_SQL)

with open('ncvoter_sample.tsv','r') as fin: 
    dr = csv.DictReader(fin, delimiter = '\t') 
    to_db = [(i["county_id"],i["county_desc"],i["county_desc"],i["status_cd"],i["voter_status_desc"],i["reason_cd"],i["voter_status_reason_desc"],i["absent_ind"],i["name_prefx_cd"],i["last_name"],i["first_name"],i["midl_name"],i["name_sufx_cd"],i["full_name_rep"],i["full_name_mail"],i["house_num"],i["half_code"],i["street_dir"],i["street_name"],i["street_type_cd"],i["street_sufx_cd"],i["unit_designator"],i["unit_num"],i["res_city_desc"],i["state_cd"],i["zip_code"],i["res_street_address"],i["res_city_state_zip"],i["mail_addr1"],i["mail_addr2"],i["mail_addr3"],i["mail_addr4"],i["mail_city"],i["mail_state"],i["mail_zipcode"],i["mail_city_state_zip"],i["area_cd"],i["phone_num"],i["full_phone_number"],i["drivers_lic"],i["race_code"],i["race_desc"],i["ethnic_code"],i["ethnic_desc"],i["party_cd"],i["party_desc"],i["sex_code"],i["sex"],i["birth_age"],i["birth_place"],i["registr_dt"],i["precinct_abbrv"],i["precinct_desc"],i["municipality_abbrv"],i["municipality_desc"],i["ward_abbrv"],i["ward_desc"],i["cong_dist_abbrv"],i["cong_dist_desc"],i["super_court_abbrv"],i["super_court_desc"],i["judic_dist_abbrv"],i["judic_dist_desc"],i["nc_senate_abbrv"],i["nc_senate_desc"],i["nc_house_abbrv"],i["nc_house_desc"],i["county_commiss_abbrv"],i["county_commiss_desc"],i["township_abbrv"],i["township_desc"],i["school_dist_abbrv"],i["school_dist_desc"],i["fire_dist_abbrv"],i["fire_dist_desc"],i["water_dist_abbrv"],i["water_dist_desc"],i["sewer_dist_abbrv"],i["sewer_dist_desc"],i["sanit_dist_abbrv"],i["sanit_dist_desc"],i["rescue_dist_abbrv"],i["rescue_dist_desc"],i["munic_dist_abbrv"],i["munic_dist_desc"],i["dist_1_abbrv"],i["dist_1_desc"],i["dist_2_abbrv"],i["dist_2_desc"],i["confidential_ind"],i["age"],i["ncid"],i["vtd_abbrv"],i["vtd_desc"]) for i in dr]
sqlite_cursor.executemany("INSERT INTO ncvoters (county_id,county_desc,county_desc,status_cd,voter_status_desc,reason_cd,voter_status_reason_desc,absent_ind,name_prefx_cd,last_name,first_name,midl_name,name_sufx_cd,full_name_rep,full_name_mail,house_num,half_code,street_dir,street_name,street_type_cd,street_sufx_cd,unit_designator,unit_num,res_city_desc,state_cd,zip_code,res_street_address,res_city_state_zip,mail_addr1,mail_addr2,mail_addr3,mail_addr4,mail_city,mail_state,mail_zipcode,mail_city_state_zip,area_cd,phone_num,full_phone_number,drivers_lic,race_code,race_desc,ethnic_code,ethnic_desc,party_cd,party_desc,sex_code,sex,birth_age,birth_place,registr_dt,precinct_abbrv,precinct_desc,municipality_abbrv,municipality_desc,ward_abbrv,ward_desc,cong_dist_abbrv,cong_dist_desc,super_court_abbrv,super_court_desc,judic_dist_abbrv,judic_dist_desc,nc_senate_abbrv,nc_senate_desc,nc_house_abbrv,nc_house_desc,county_commiss_abbrv,county_commiss_desc,township_abbrv,township_desc,school_dist_abbrv,school_dist_desc,fire_dist_abbrv,fire_dist_desc,water_dist_abbrv,water_dist_desc,sewer_dist_abbrv,sewer_dist_desc,sanit_dist_abbrv,sanit_dist_desc,rescue_dist_abbrv,rescue_dist_desc,munic_dist_abbrv,munic_dist_desc,dist_1_abbrv,dist_1_desc,dist_2_abbrv,dist_2_desc,confidential_ind,age,ncid,vtd_abbrv,vtd_desc) VALUES (?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?,?,?,?, ?);", to_db)

with open('precinct_votes.tsv','r') as fin: 
    dr = csv.DictReader(fin, delimiter = '\t') 
    to_db = [(i["county"], i["precinct"], i["total_votes"], i["romney_percentage"]) for i in dr]
sqlite_cursor.executemany("INSERT INTO precinct_votes (county, precinct,total_votes,romney_percentage) VALUES (?, ?,?,?);", to_db)

db_sqlite.commit()
end = time.time()
print("Loading data in SQLite : " +str(end - start))

How do we load data in DuckDB?

The SQL Way:

https://duckdb.org/docs/sql/statements/create_table


In [ ]:
import duckdb
start = time.time()
duck_conn = duckdb.connect('voters_duck.db')
duck_cursor = duck_conn.cursor()
duck_cursor.execute("CREATE TABLE ncvoters AS SELECT * FROM read_csv_auto ('ncvoter_sample.tsv')")
duck_cursor.execute("CREATE TABLE precinct_votes AS SELECT * FROM read_csv_auto ('precinct_votes.tsv')")
end = time.time()
print("Loading data in DuckDB : " +str(end - start))


How do we load data in DuckDB?

The Relational API Way:

https://github.com/cwida/duckdb/blob/master/examples/python/duckdb-python.py

Obs: The relational API is a work in progress, so expect the unexpected.

In [ ]:
start = time.time()
duck_conn.from_csv_auto("ncvoter_sample.tsv").create("ncvoters_rapi")
duck_conn.from_csv_auto("precinct_votes.tsv").create("precinct_votes_rapi")
end = time.time()
print("Loading data in DuckDB (RAPI): " +str(end - start))


# **Queries**

Your task is to answer the next three queries on both sqlite and duckdb (because they have the same sql syntax ;-) ) and benchmark them. After it, explain why the times to answer the queries differ in those 3 systems.

**Query Q1**: How many people voted in the elections?

*TIP_1 : First step is to issue a Query, how do you make the cursor issue a query? (Look on previous exercise or at https://duckdb.org/docs/api/python)*

*TIP_2 : Now is time to write the query. You must check the if the voters have an 'Active' status (i.e., ncvoters.status_cd='A')*

*TIP_3 : Finally, you have to print the results and compare them for correctness*



In [ ]:
db_sqlite = sqlite3.connect('voters_sqlite.db') 
sqlite_cursor = db_sqlite.cursor() 
#Fill Query here!
query_01 = "SELECT COUNT(*) FROM ncvoters WHERE ncvoters.status_cd=\'A\'"
def query_01_pandas_df(df):
	pandas_query = df[["county_id"]][(df.status_cd == 'A')]
	result = (pandas_query.county_id).count()
	print (result)

def query_01_pandas():
	query_01_pandas_df(ncvoters_pandas)

#Implement this method
def query_01_sqlite():
	sqlite_cursor.execute(query_01)
	result = sqlite_cursor.fetchone()
	print(result)
 
#Implement this method
def query_01_duckdb(): 
	duck_cursor.execute(query_01)
	result = duck_cursor.fetchone()
	print(result)

def query_01_duckdb_rapi():
	ncvoters_duck = duck_conn.table("ncvoters_rapi")
	print(ncvoters_duck.filter('status_cd = \'A\'').aggregate('count(*)'))
 
start = time.time()
query_01_pandas()
end = time.time()
print("Query in Pandas : " +str(end - start))

start = time.time()
query_01_sqlite()
end = time.time()
print("Query in SQLite : " +str(end - start))

start = time.time()
query_01_duckdb()
end = time.time()
print("Query in DuckDB : " +str(end - start))

start = time.time()
query_01_duckdb_rapi()
end = time.time()
print("Query in DuckDB (RAPI): " +str(end - start))

**Query Q2**: Count all the people from 70 to 80 and that voted in the election. (i.e., x ppl with 70 yo, y ppl with 71, ...)

> Indented block



*TIP : Group by query with having clause.*

In [ ]:
#Fill Query Here!
query_02 = "SELECT birth_age, COUNT(*) FROM ncvoters GROUP BY birth_age HAVING birth_age > 70 and birth_age <= 80"

def query_02_pandas():
	result = ncvoters_pandas.groupby('birth_age').agg({'county_id': 'count'})
	print(result[(result.county_id.index > 70) & (result.county_id.index <= 80)])

#Implement this method
def query_02_sqlite():
	sqlite_cursor.execute(query_02)
	result = sqlite_cursor.fetchall()
	print(result)

#Implement this method
def query_02_duckdb():
	duck_cursor.execute(query_02)
	result = duck_cursor.fetchall()
	print(result)
 
 #Implement this method
def query_02_duckdb_rapi():
	ncvoters_duck = duck_conn.table("ncvoters_rapi")
	print(ncvoters_duck.filter('birth_age > 70 and birth_age <= 80').aggregate("count(*) as amount,birth_age"))
 
start = time.time()
query_02_pandas()
end = time.time()
print("Query in Pandas : " +str(end - start))

start = time.time()
query_02_sqlite()
end = time.time()
print("Query in SQLite : " +str(end - start))

start = time.time()
query_02_duckdb()
end = time.time()
print("Query in DuckDB : " +str(end - start))

start = time.time()
query_02_duckdb_rapi()
end = time.time()
print("Query in DuckDB (Rapi): " +str(end - start))

**Query Q3**: 

**Part 1 -**What are the three counties with most male, white men over 40 and are they majority democrats or republicans?

**Part 2 -** Return the data as a dataframe, so it can be used in a ML pipeline.

In [ ]:
#Fill Query Here!
query_03_sql = """SELECT county_desc, MAX(romney_percentage), COUNT(*) AS total_white
FROM precinct_votes
INNER JOIN ncvoters ON ncvoters.county_desc=precinct_votes.county
WHERE ncvoters.status_cd=\'A\' AND sex_code = \'M\'
  AND race_code = \'W\'
GROUP BY county_desc
ORDER BY total_white DESC
LIMIT 3"""

def query_03_pandas():
	pandas_query = ncvoters_pandas[["county_desc", "status_cd", "sex_code", "race_code", "birth_age"]][(ncvoters_pandas.status_cd == 'A')&(ncvoters_pandas.sex_code == 'M') &(ncvoters_pandas.race_code == 'W')].set_index("county_desc").join(precinctvotes_pandas.set_index("county"),on="county_desc")
	res = pandas_query.groupby(["county_desc"]).agg({'status_cd': 'count','romney_percentage':'max'}).reset_index(drop=True).sort_values("status_cd", ascending=False).iloc[0:3]
	print(res)
 
 #Implement this method!
def query_03_sqlite():
	result = pandas.read_sql_query(query_03_sql, db_sqlite)
	print(result)

 #Implement this method!
def query_03_duckdb():
	duck_cursor.execute(query_03_sql)
	result = duck_cursor.fetchdf()
	print(result)
 
def query_03_duckdb_rapi():
	ncvoters_duck = duck_conn.table("ncvoters")
	precinct_duck = duck_conn.table("precinct_votes")
	print(ncvoters_duck.filter("sex_code = 'M' AND race_code = 'W' AND status_cd = 'A'").join(precinct_duck,"ncvoters.county_desc=precinct_votes.county").aggregate("count(*) as total_white,MAX(romney_percentage),county_desc").order('total_white DESC').limit(3))

start = time.time()
query_03_pandas()
end = time.time()
print("Query in Pandas : " +str(end - start))

start = time.time()
query_03_sqlite()
end = time.time()
print("Query in SQLite : " +str(end - start))

start = time.time()
query_03_duckdb()
end = time.time()
print("Query in DuckDB : " +str(end - start))


start = time.time()
query_03_duckdb_rapi()
end = time.time()
print("Query in DuckDB (RAPI): " +str(end - start))

# **Transactions**

Here we will do a simple data cleaning task. 

Knowing that the minimum age to vote in the US is 18 years old and that the oldest person that in the US during these elections was 120 years old, can you detect any anomalies?

Correct the data by removing dirty tuples from SQLite and DuckDB. 

In [ ]:
query_detect_anomalies = """
SELECT count(*) FROM ncvoters where birth_age < 18 or birth_age > 120
"""

duck_cursor.execute(query_detect_anomalies)
result = duck_cursor.fetchone()
print(result)


In [ ]:
#Fill query here
drop_anomalies = """DELETE FROM ncvoters WHERE birth_age < 18 OR birth_age > 120""" 

def query_04_pandas():
	birth_age_gt_18 = ncvoters_pandas['birth_age'] >= 18
	birth_age_lt_120 =  ncvoters_pandas['birth_age'] <= 120
	result = ncvoters_pandas[birth_age_gt_18 & birth_age_lt_120]
	result.to_csv('result.tsv.tmp', sep='\t')

# Implement this method
def query_04_sqlite():
	sqlite_cursor.execute(drop_anomalies)

# Implement this method
def query_04_duckdb():
	duck_cursor.execute(drop_anomalies)
 
start = time.time()
query_04_pandas()
end = time.time()
print("Transaction in Pandas : " +str(end - start))

start = time.time()
query_04_sqlite()
end = time.time()
print("Transaction in SQLite : " +str(end - start))

start = time.time()
query_04_duckdb()
end = time.time()
print("Transaction in DuckDB : " +str(end - start))